# Data Preprocessing Including Categorical Encoding

This Jupyter Notebook provides the preprocessing steps including handling of both numeric and categorical data for the Telco Customer Churn dataset.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

## Load the Data

In [2]:
# Load the dataset
data_path = './WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(data_path)

## Initial Data Exploration

In [3]:
# Display the first few rows of the dataframe
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# Display data types and missing values
print(df.dtypes)
print(df.isnull().sum())

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Mo

## Remove Irrelevant Columns

In [5]:
# Assuming 'customerID' is not needed for analysis
df.drop('customerID', axis=1, inplace=True)

## Handling Missing Values

In [6]:
# Convert 'TotalCharges' from object to numeric and handle missing/incorrect values
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan).astype(float)
df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)

## Encoding Categorical Variables

### Convert binary categorical variables (Yes/No) to numeric (0/1)

In [7]:
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in binary_cols:
    df[col] = df[col].apply(lambda x: 1 if x == 'Yes' else 0)

### Label Encoding

In [8]:
# Label encoding for binary categories
label_encoder = LabelEncoder()
df['gender'] = label_encoder.fit_transform(df['gender'])

### One-Hot Encoding

In [9]:
# One-hot encoding for nominal categories
categorical_cols = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
                    'PaymentMethod']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

## Binning 'tenure' into Categorical Bins

In [10]:
bins = [0, 12, 24, 60, 120]
labels = ['0-1 year', '1-2 years', '2-5 years', '5+ years']
df['tenure_bins'] = pd.cut(df['tenure'], bins=bins, labels=labels, right=False)

## Normalizing Numeric Data

In [11]:
# Standardizing numeric features to have a mean of 0 and a standard deviation of 1
scaler = StandardScaler()
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [12]:
print(df.head())
print(df.dtypes)

   gender  SeniorCitizen  Partner  Dependents    tenure  PhoneService  \
0       0              0        1           0 -1.277445             0   
1       1              0        0           0  0.066327             1   
2       1              0        0           0 -1.236724             1   
3       1              0        0           0  0.514251             0   
4       0              0        0           0 -1.236724             1   

   PaperlessBilling  MonthlyCharges  TotalCharges  Churn  ...  \
0                 1       -1.160323     -0.994971      0  ...   
1                 0       -0.259629     -0.173876      0  ...   
2                 1       -0.362660     -0.960399      1  ...   
3                 0       -0.746535     -0.195400      0  ...   
4                 1        0.197365     -0.941193      1  ...   

   StreamingTV_No internet service  StreamingTV_Yes  \
0                            False            False   
1                            False            False   
2    

## Save the Preprocessed Data

In [13]:
# Save the cleaned and encoded dataframe for further analysis
df.to_csv('./processed_telco_churn.csv', index=False)